In [1]:

from databaker.framework import *
from databakerUtils.writers import v4Writer


# name the input and output file
inputfile = 'headlinewellbeinglocalauthorityupdate2016to2017 (3).xls'
outputfile ='v4_wellbeingInitial.csv'

# load the tabs we want
tabs = loadxlstabs(inputfile)

Loading headlinewellbeinglocalauthorityupdate2016to2017 (3).xls which has size 1042432 bytes
Table names: ['Contents', 'Life Satisfaction - Means ', 'Life Satisfaction - Thresholds', 'Worthwhile - Means', 'Worthwhile - Thresholds', 'Happy - Means', 'Happy - Thresholds', 'Anxiety - Means', 'Anxiety - Thresholds']


In [2]:

# Filter to just the tabs that we want
tabs = [tab for tab in tabs if "Thresholds" in tab.name]

for tab in tabs:
    print(tab.name)
    

Life Satisfaction - Thresholds
Worthwhile - Thresholds
Happy - Thresholds
Anxiety - Thresholds


In [3]:

import pandas as pd

conversionsegments = []

for tab in tabs:
    
    observations = tab.excel_ref("C8").expand(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace()
    
    key = tab.excel_ref("O").filter(contains_string("Key"))
    unwanted = key.expand(DOWN).expand(RIGHT)
    
    observations = observations - unwanted
    
    measure = tab.name.split(" ")[0].strip()
    
    indicators = tab.excel_ref("C7").expand(RIGHT).is_not_blank().is_not_whitespace()
    
    geography = tab.excel_ref("A8").expand(DOWN).is_not_blank().is_not_whitespace()
    geographyLabels = tab.excel_ref("B8").expand(DOWN).is_not_blank().is_not_whitespace()
    
    time = tab.excel_ref("C5").expand(RIGHT).is_not_blank().is_not_whitespace()
    
    
    dimensions = [
        HDim(indicators, "Indicators", DIRECTLY, ABOVE),
        HDim(time, TIME, CLOSEST, LEFT),
        HDim(geography, GEOG, DIRECTLY, LEFT),
        HDim(geographyLabels, "geoLabels", DIRECTLY, LEFT), # NOTE - temporary dimension
        HDimConst("Measure", measure)
    ]
    
    conversionsegment = ConversionSegment(tab, dimensions, observations).topandas()
    
    conversionsegments.append(conversionsegment)

    
    

allData = pd.concat(conversionsegments)    

df = v4Writer(outputfile, allData, asFrame=True)

df

TIMEUNIT=''
TIMEUNIT=''
TIMEUNIT=''
TIMEUNIT=''
Extracting data structured as v4.


,V4_0,Data_Marking,Time_codelist,Time,Geography_codelist,Geography,Indicators_codelist,Indicators,geoLabels_codelist,geoLabels,Measure_codelist,Measure
0,6.51,NaN,,2011/12,K02000001,,,0-4,,UNITED KINGDOM,,Life
1,17.44,NaN,,2011/12,K02000001,,,5-6,,UNITED KINGDOM,,Life
2,49.83,NaN,,2011/12,K02000001,,,7-8,,UNITED KINGDOM,,Life
3,26.22,NaN,,2011/12,K02000001,,,9-10,,UNITED KINGDOM,,Life
4,5.71,NaN,,2012/13*,K02000001,,,0-4,,UNITED KINGDOM,,Life
5,17.1,NaN,,2012/13*,K02000001,,,5-6,,UNITED KINGDOM,,Life
6,51.16,NaN,,2012/13*,K02000001,,,7-8,,UNITED KINGDOM,,Life
7,26.03,NaN,,2012/13*,K02000001,,,9-10,,UNITED KINGDOM,,Life
8,5.54,NaN,,2013/14*,K02000001,,,0-4,,UNITED KINGDOM,,Life
9,15.92,NaN,,2013/14*,K02000001,,,5-6,,UNITED KINGDOM,,Life


In [4]:

# Post processing

# GEOGRAPHY
def tidyCase(cell):
    return cell.title()

df["Geography"] = df["geoLabels"]
df = df.drop("geoLabels", axis=1)
df = df.drop("geoLabels_codelist", axis=1)

df["Geography"] = df["Geography"].apply(tidyCase)

df

,V4_0,Data_Marking,Time_codelist,Time,Geography_codelist,Geography,Indicators_codelist,Indicators,Measure_codelist,Measure
0,6.51,NaN,,2011/12,K02000001,United Kingdom,,0-4,,Life
1,17.44,NaN,,2011/12,K02000001,United Kingdom,,5-6,,Life
2,49.83,NaN,,2011/12,K02000001,United Kingdom,,7-8,,Life
3,26.22,NaN,,2011/12,K02000001,United Kingdom,,9-10,,Life
4,5.71,NaN,,2012/13*,K02000001,United Kingdom,,0-4,,Life
5,17.1,NaN,,2012/13*,K02000001,United Kingdom,,5-6,,Life
6,51.16,NaN,,2012/13*,K02000001,United Kingdom,,7-8,,Life
7,26.03,NaN,,2012/13*,K02000001,United Kingdom,,9-10,,Life
8,5.54,NaN,,2013/14*,K02000001,United Kingdom,,0-4,,Life
9,15.92,NaN,,2013/14*,K02000001,United Kingdom,,5-6,,Life


In [5]:

# MEASURE
df["Measure_codelist"] = df["Measure"].map(lambda x: x.lower())
df

,V4_0,Data_Marking,Time_codelist,Time,Geography_codelist,Geography,Indicators_codelist,Indicators,Measure_codelist,Measure
0,6.51,NaN,,2011/12,K02000001,United Kingdom,,0-4,life,Life
1,17.44,NaN,,2011/12,K02000001,United Kingdom,,5-6,life,Life
2,49.83,NaN,,2011/12,K02000001,United Kingdom,,7-8,life,Life
3,26.22,NaN,,2011/12,K02000001,United Kingdom,,9-10,life,Life
4,5.71,NaN,,2012/13*,K02000001,United Kingdom,,0-4,life,Life
5,17.1,NaN,,2012/13*,K02000001,United Kingdom,,5-6,life,Life
6,51.16,NaN,,2012/13*,K02000001,United Kingdom,,7-8,life,Life
7,26.03,NaN,,2012/13*,K02000001,United Kingdom,,9-10,life,Life
8,5.54,NaN,,2013/14*,K02000001,United Kingdom,,0-4,life,Life
9,15.92,NaN,,2013/14*,K02000001,United Kingdom,,5-6,life,Life


In [6]:

# TIME
df["Time_codelist"] = df["Time"].map(lambda x: x.lower())
df

,V4_0,Data_Marking,Time_codelist,Time,Geography_codelist,Geography,Indicators_codelist,Indicators,Measure_codelist,Measure
0,6.51,NaN,2011/12,2011/12,K02000001,United Kingdom,,0-4,life,Life
1,17.44,NaN,2011/12,2011/12,K02000001,United Kingdom,,5-6,life,Life
2,49.83,NaN,2011/12,2011/12,K02000001,United Kingdom,,7-8,life,Life
3,26.22,NaN,2011/12,2011/12,K02000001,United Kingdom,,9-10,life,Life
4,5.71,NaN,2012/13*,2012/13*,K02000001,United Kingdom,,0-4,life,Life
5,17.1,NaN,2012/13*,2012/13*,K02000001,United Kingdom,,5-6,life,Life
6,51.16,NaN,2012/13*,2012/13*,K02000001,United Kingdom,,7-8,life,Life
7,26.03,NaN,2012/13*,2012/13*,K02000001,United Kingdom,,9-10,life,Life
8,5.54,NaN,2013/14*,2013/14*,K02000001,United Kingdom,,0-4,life,Life
9,15.92,NaN,2013/14*,2013/14*,K02000001,United Kingdom,,5-6,life,Life


In [7]:

# INDICATOR
indicatorLookup = {
        "9-10":"Very good",
        "0-1":"Very good",
        "7-8":"Good",
        "2-3":"Good",
        "5-6":"Fair",
        "5-4":"Fair",
        "4-5":"Fair",
        "0-4":"Poor",
        "6-10":"Poor"
    }
df["Indicators_codelist"] = df["Indicators"].map(lambda x: indicatorLookup[str(x).strip()])
df

,V4_0,Data_Marking,Time_codelist,Time,Geography_codelist,Geography,Indicators_codelist,Indicators,Measure_codelist,Measure
0,6.51,NaN,2011/12,2011/12,K02000001,United Kingdom,Poor,0-4,life,Life
1,17.44,NaN,2011/12,2011/12,K02000001,United Kingdom,Fair,5-6,life,Life
2,49.83,NaN,2011/12,2011/12,K02000001,United Kingdom,Good,7-8,life,Life
3,26.22,NaN,2011/12,2011/12,K02000001,United Kingdom,Very good,9-10,life,Life
4,5.71,NaN,2012/13*,2012/13*,K02000001,United Kingdom,Poor,0-4,life,Life
5,17.1,NaN,2012/13*,2012/13*,K02000001,United Kingdom,Fair,5-6,life,Life
6,51.16,NaN,2012/13*,2012/13*,K02000001,United Kingdom,Good,7-8,life,Life
7,26.03,NaN,2012/13*,2012/13*,K02000001,United Kingdom,Very good,9-10,life,Life
8,5.54,NaN,2013/14*,2013/14*,K02000001,United Kingdom,Poor,0-4,life,Life
9,15.92,NaN,2013/14*,2013/14*,K02000001,United Kingdom,Fair,5-6,life,Life


In [8]:

# get rid of nan values
df.fillna("", inplace=True)
df

,V4_0,Data_Marking,Time_codelist,Time,Geography_codelist,Geography,Indicators_codelist,Indicators,Measure_codelist,Measure
0,6.51,,2011/12,2011/12,K02000001,United Kingdom,Poor,0-4,life,Life
1,17.44,,2011/12,2011/12,K02000001,United Kingdom,Fair,5-6,life,Life
2,49.83,,2011/12,2011/12,K02000001,United Kingdom,Good,7-8,life,Life
3,26.22,,2011/12,2011/12,K02000001,United Kingdom,Very good,9-10,life,Life
4,5.71,,2012/13*,2012/13*,K02000001,United Kingdom,Poor,0-4,life,Life
5,17.1,,2012/13*,2012/13*,K02000001,United Kingdom,Fair,5-6,life,Life
6,51.16,,2012/13*,2012/13*,K02000001,United Kingdom,Good,7-8,life,Life
7,26.03,,2012/13*,2012/13*,K02000001,United Kingdom,Very good,9-10,life,Life
8,5.54,,2013/14*,2013/14*,K02000001,United Kingdom,Poor,0-4,life,Life
9,15.92,,2013/14*,2013/14*,K02000001,United Kingdom,Fair,5-6,life,Life
